# Optymalizacja trasy tramwajowej

Ten notebook zawiera wizualizację i optymalizację trasy tramwajowej w Krakowie.

In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

import geopandas as gpd
from src.optimization.route_optimizer import RouteOptimizer, RouteConstraints
from src.visualization.route_visualizer import RouteVisualizer
from scripts.sourcing_data import TramData, OpenStreetMapData
from shapely.geometry import Point, LineString
import folium
import logging
import numpy as np
import random

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

INFO:scripts.sourcing_data:Logging initialized.


## Pobieranie danych

In [2]:
# Pobieranie danych z MPK i OpenStreetMap
tram_data = TramData()
osm_data = OpenStreetMapData()

# Konwersja danych do GeoDataFrame
stops_gdf = gpd.GeoDataFrame(
    tram_data.stops_df,
    geometry=[Point(lon, lat) for lat, lon in zip(tram_data.stops_df['latitude'], tram_data.stops_df['longitude'])],
    crs="EPSG:4326"
)

# Konwersja linii
lines_data = []
for line, coords in tram_data.mpk_sourcing.lines_stops_coordinates.items():
    if coords:  # sprawdzenie czy lista nie jest pusta
        line_geom = LineString([(lon, lat) for lat, lon in coords])
        lines_data.append({
            'line': line,
            'geometry': line_geom
        })
lines_gdf = gpd.GeoDataFrame(lines_data, crs="EPSG:4326")

INFO:scripts.sourcing_data:Initializing TramData...
INFO:scripts.sourcing_data:Starting fetch for stops and passages...
INFO:scripts.sourcing_data:Fetching stops data from API...
INFO:scripts.sourcing_data:Successfully fetched stops data.
INFO:scripts.sourcing_data:Fetching passage data for stop: 458
INFO:scripts.sourcing_data:Fetching passage data for stop: 457
INFO:scripts.sourcing_data:Fetching passage data for stop: 126


Loading config from: e:\projects\krakow-tram-optimization\config.ini


INFO:scripts.sourcing_data:Fetching passage data for stop: 3038
INFO:scripts.sourcing_data:Fetching passage data for stop: 2691
INFO:scripts.sourcing_data:Fetching passage data for stop: 2690
INFO:scripts.sourcing_data:Fetching passage data for stop: 747
INFO:scripts.sourcing_data:Fetching passage data for stop: 824
INFO:scripts.sourcing_data:Fetching passage data for stop: 3837
INFO:scripts.sourcing_data:Fetching passage data for stop: 746
INFO:scripts.sourcing_data:Fetching passage data for stop: 3836
INFO:scripts.sourcing_data:Fetching passage data for stop: 3643
INFO:scripts.sourcing_data:Fetching passage data for stop: 3647
INFO:scripts.sourcing_data:Fetching passage data for stop: 3176
INFO:scripts.sourcing_data:Fetching passage data for stop: 744
INFO:scripts.sourcing_data:Fetching passage data for stop: 3646
INFO:scripts.sourcing_data:Fetching passage data for stop: 2821
INFO:scripts.sourcing_data:Fetching passage data for stop: 311
INFO:scripts.sourcing_data:Fetching passage d

## Wizualizacja istniejących danych

In [ ]:
# Inicjalizacja wizualizatora
visualizer = RouteVisualizer(osm_data.buildings_df, osm_data.streets_df)

# Tworzenie podstawowej mapy
m = visualizer.create_base_map()

# Dodawanie istniejących linii
for _, row in lines_gdf.iterrows():
    coords = [(lat, lon) for lon, lat in row.geometry.coords]
    visualizer.plot_route(coords, m, f"Linia {row['line']}", color='gray')

# Dodawanie przystanków
for _, row in stops_gdf.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=4,
        color='red',
        fill=True,
        popup=f'Przystanek {row.name}',
        name=f'Przystanek {row.name}'
    ).add_to(m)

m

## Optymalizacja trasy z wizualizacją procesu

In [ ]:
# Konfiguracja ograniczeń
constraints = RouteConstraints(
    min_distance_between_stops=300,  # 300m między przystankami
    max_distance_between_stops=1000,  # 1000m między przystankami
    max_angle=45,  # maksymalny kąt zakrętu
    min_route_length=5,  # minimalna liczba przystanków
    max_route_length=20,  # maksymalna liczba przystanków
    min_total_length=2000,  # minimalna długość trasy
    max_total_length=10000,  # maksymalna długość trasy
    min_distance_from_buildings=5  # minimalna odległość od budynków
)

# Inicjalizacja optymalizatora
optimizer = RouteOptimizer(
    buildings_df=osm_data.buildings_df,
    streets_df=osm_data.streets_df,
    stops_df=stops_gdf,
    lines_df=lines_gdf,
    constraints=constraints,
    population_size=100,
    generations=50,
    mutation_rate=0.1,
    crossover_rate=0.8,
    min_stop_distance=300,
    max_stop_distance=800,
    population_weight=0.7,
    distance_weight=0.3
)

# Przygotowanie do wizualizacji
scores_history = []
best_route = None
best_score = float('-inf')

# Uruchomienie optymalizacji z wizualizacją
population = optimizer._create_initial_population()

for generation in range(optimizer.generations):
    # Ocena populacji
    scores = [optimizer._evaluate_route(route) for route in population]
    
    # Aktualizacja najlepszej trasy
    max_score_idx = np.argmax(scores)
    if scores[max_score_idx] > best_score:
        best_score = scores[max_score_idx]
        best_route = population[max_score_idx]
    
    scores_history.append(best_score)
    
    # Obliczenie granic obszaru
    bounds = (
        min(lon for lat, lon in best_route),
        min(lat for lat, lon in best_route),
        max(lon for lat, lon in best_route),
        max(lat for lat, lon in best_route)
    )
    
    # Generowanie mapy gęstości
    density_map = optimizer.density_calculator.get_density_map(
        grid_size=0.001,
        bounds=bounds
    )
    
    # Wizualizacja procesu
    visualizer.visualize_optimization_process(
        population=population,
        scores=scores,
        generation=generation + 1,
        best_route=best_route,
        best_score=best_score,
        bounds=bounds,
        density_map=density_map,
        update_interval=0.5
    )
    
    # Selekcja i tworzenie nowej populacji
    selected_indices = np.argsort(scores)[-optimizer.population_size//2:]
    selected = [population[i] for i in selected_indices]
    
    new_population = selected.copy()
    
    while len(new_population) < optimizer.population_size:
        parent1, parent2 = random.sample(selected, 2)
        child1, child2 = optimizer._crossover(parent1, parent2)
        
        child1 = optimizer._mutate(child1)
        child2 = optimizer._mutate(child2)
        
        if optimizer._is_valid_route(child1):
            new_population.append(child1)
        if optimizer._is_valid_route(child2) and len(new_population) < optimizer.population_size:
            new_population.append(child2)
    
    population = new_population

## Wizualizacja postępu optymalizacji

In [ ]:
visualizer.plot_optimization_progress(scores_history)